### Import

In [ ]:
import numpy as np
import xarray as xr
from datetime import *
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn.colors.xkcd_rgb as c
import cmaps
import cartopy.crs as ccrs
import cartopy.feature as cfeature

### Energy Time Series

In [ ]:
ds_sfc = xr.open_mfdataset(f'/data/mlcloud/ch995334/VVM/DATA/pbl_control_dt/archive/pbl_control_dt.C.Surface*')

In [ ]:
ds_rad = xr.open_mfdataset(f'/data/mlcloud/ch995334/VVM/DATA/pbl_control_dt/archive/pbl_control_dt.L.Radiation*')

In [ ]:
# Surface flux of potential temperature
da_THF = ds_sfc.wth
# Surface flux of potential water vapor
da_QV  = ds_sfc.wqv

In [ ]:
# Downward SW @ TOA
da_dsw = ds_rad.fdswtoa

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.grid(':', linewidth=0.5)
lsw     = ax.plot(np.arange(da_dsw.shape[0]), da_dsw[:, 0, 0], 'k', linewidth=3, label='SW @ TOA')
lth     = ax.plot(np.arange(da_THF.shape[0]), da_THF[:, 0, 0]*1004, c['reddish'], linewidth=3, label=r'$\theta$ flux')
lqv     = ax.plot(np.arange(da_QV.shape[0]), da_QV[:, 0, 0], c['bluish'], linewidth=3, label=r'qv flux')
ax.set_xlim(0, 1441)
ax.set_xlabel('LST', fontsize=16)
ax.set_xticks((np.array([6, 12, 18, 24])-5)*60)
ax.set_xticklabels([f'{i}:00' for i in range(6, 25, 6)], fontsize=14)
ax.set_ylim(-15, 800)
ax.set_ylabel('(W/$m^{2}$)', fontsize=16)
ax.set_yticks(np.arange(0, 800.1, 100))
ax.set_yticklabels(np.arange(0, 800.1, 100).astype(int), fontsize=14)
ax.set_title('PBL_control', loc='left', fontsize=16)
ax.set_title('(1, 1)', loc='right', fontsize=16)
ax.legend(fontsize=16)
plt.savefig('/data/mlcloud/ch995334/hw1/tseries_dt5.png', facecolor='w', bbox_inches='tight', dpi=400)

### var profile

In [ ]:
ds_dyn = xr.open_mfdataset(f'/data/mlcloud/ch995334/VVM/DATA/pbl_control_dt/archive/pbl_control_dt.L.Dynamic*')

In [ ]:
ds_thm = xr.open_mfdataset(f'/data/mlcloud/ch995334/VVM/DATA/pbl_control_dt/archive/pbl_control_dt.L.Thermodynamic*')

In [ ]:
da_u   = ds_dyn.u
da_v   = ds_dyn.v
da_w   = ds_dyn.w

In [ ]:
da_t   = ds_thm.th
da_qv  = ds_thm.qv

In [ ]:
level  = ds_dyn.lev

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(10, 6), sharey=True, gridspec_kw={'wspace':0.25})
# Grid lines for all subplots
ax[0].grid(':', linewidth=0.5)
ax[1].grid(':', linewidth=0.5)
ax[2].grid(':', linewidth=0.5)
ax[3].grid(':', linewidth=0.5)
# Required time stamp (LST)
tstamp = np.insert((np.array([6, 12, 18, 24])-5)*60, 0, 0)
# Line label
llabel = ['Init.', '0600', '1200', '1800', '0000']
# Line color
lcolor = ['black', 'bluish', 'sage', 'gold', 'scarlet']
for tt in range(tstamp.shape[0]):
    # U plot
    ax[0].plot(da_u[tstamp[tt], :, 0, 0], level, linewidth=3, color=c[lcolor[tt]])
    # V plot
    ax[1].plot(da_v[tstamp[tt], :, 0, 0], level, linewidth=3, color=c[lcolor[tt]])
    # Theta plot
    ax[2].plot(da_t[tstamp[tt], :, 0, 0], level, linewidth=3, color=c[lcolor[tt]])
    # qv plot
    ax[3].plot(da_qv[tstamp[tt], :, 0, 0], level, linewidth=3, color=c[lcolor[tt]], label=llabel[tt])
# Info. settings
ax[3].legend(fontsize=12)
ax[0].set_title('U (m/s)', fontsize=16)
ax[0].set_ylim(0, 2)
ax[0].set_yticks(np.arange(0, 2.1, 0.25))
ax[0].set_yticklabels(np.arange(0, 2.1, 0.25), fontsize=12)
ax[0].set_ylabel('Height (km)', fontsize=14)
ax[0].set_xlim(-2.5, 2.5)
ax[0].set_xticks(np.arange(-2, 2.1, 2))
ax[0].set_xticklabels(np.arange(-2, 2.1, 2).astype(int), fontsize=12)
ax[1].set_title('V (m/s)', fontsize=16)
ax[1].set_xlim(-2.5, 2.5)
ax[1].set_xticks(np.arange(-2, 2.1, 2))
ax[1].set_xticklabels(np.arange(-2, 2.1, 2).astype(int), fontsize=12)
ax[2].set_title(r'$\theta$ (K)', fontsize=16)
ax[2].set_xlim(280, 310)
ax[2].set_xticks(np.array([290, 300]))
ax[2].set_xticklabels(np.array([290, 300]).astype(int), fontsize=12)
ax[3].set_title('qv (kg/kg)', fontsize=16)
ax[3].set_xlim(0, 2e-5)
ax[3].set_xticks(np.arange(0, 2.5e-5, 1e-5))
ax[3].set_xticklabels(np.arange(0, 2.5e-5, 1e-5), fontsize=14)
plt.savefig('/data/mlcloud/ch995334/hw1/4profile_dt5.png', facecolor='w', bbox_inches='tight', dpi=400)

### 2D map @ 1000m (-> animation)

In [ ]:
# 1000m = level 26 (python idx 25)
pi1000 = 0.967    #T=theta*pi

In [ ]:
# Field 1000m
u1000 = da_u.isel(lev=25)
v1000 = da_v.isel(lev=25)
w1000 = da_w.isel(lev=25)
qv1000= da_qv.isel(lev=25)
T1000 = da_t.isel(lev=25)*pi1000

In [ ]:
datetest = datetime(2024, 1, 1, 5, 0, 0)
datenew  = datetest+timedelta(minutes=1)
print(datetest, datenew)
print(datenew.strftime('%H:%M'))

In [ ]:
# Wind field 1000m
time_now    = datetime(2024, 1, 1, 5, 0, 0)
for tt in range(u1000.shape[0]):
    fig, ax = plt.subplots(figsize=(6, 6))
    imw     = ax.imshow(w1000[tt, ...], vmin=-2, vmax=2, origin='lower', cmap=cmaps.MPL_RdBu)
    ax.quiver(np.arange(128)[::4], np.arange(128)[::4], u1000[tt, ::4, ::4], v1000[tt, ::4, ::4])
    cax     = fig.add_axes([ax.get_position().x1+0.015, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar    = fig.colorbar(imw, cax=cax, extend='both')
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label(label='w (m/s)', fontsize=14)
    ax.set_xticks(np.arange(0, 128.1, 40))
    ax.set_xticklabels(np.arange(0, 128.1, 40)*0.2, fontsize=14)
    ax.set_xlabel('km', fontsize=14)
    ax.set_yticks(np.arange(0, 128.1, 40))
    ax.set_yticklabels(np.arange(0, 128.1, 40)*0.2, fontsize=14)
    ax.set_ylabel('km', fontsize=14)
    ax.set_title('Wind Field @ 1000m', loc='left', fontsize=16)
    ax.set_title(time_now.strftime('%H:%M'), loc='right', fontsize=16)
    plt.savefig(f'/data/mlcloud/ch995334/hw1/wind_animation/dt5_{time_now.strftime("%d%H%M")}.png', 
                facecolor='w', bbox_inches='tight', dpi=400)
    time_now= time_now+timedelta(minutes=1)

In [ ]:
# Temp. field 1000m
time_now    = datetime(2024, 1, 1, 5, 0, 0)
for tt in range(u1000.shape[0]):
    fig, ax = plt.subplots(figsize=(6, 6))
    imT     = ax.imshow(T1000[tt, ...], vmin=286.6, vmax=287.6, origin='lower', cmap=cmaps.cmocean_matter)
    cax     = fig.add_axes([ax.get_position().x1+0.015, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar    = fig.colorbar(imT, cax=cax, extend='both')
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label(label='T (K)', fontsize=14)
    ax.set_xticks(np.arange(0, 128.1, 40))
    ax.set_xticklabels(np.arange(0, 128.1, 40)*0.2, fontsize=14)
    ax.set_xlabel('km', fontsize=14)
    ax.set_yticks(np.arange(0, 128.1, 40))
    ax.set_yticklabels(np.arange(0, 128.1, 40)*0.2, fontsize=14)
    ax.set_ylabel('km', fontsize=14)
    ax.set_title('Temp. Field @ 1000m', loc='left', fontsize=16)
    ax.set_title(time_now.strftime('%H:%M'), loc='right', fontsize=16)
    plt.savefig(f'/data/mlcloud/ch995334/hw1/T_animation/dt5_{time_now.strftime("%d%H%M")}.png', 
                facecolor='w', bbox_inches='tight', dpi=400)
    time_now= time_now+timedelta(minutes=1)

In [ ]:
# qv field 1000m
time_now    = datetime(2024, 1, 1, 5, 0, 0)
for tt in range(u1000.shape[0]):
    fig, ax = plt.subplots(figsize=(6, 6))
    imT     = ax.imshow(T1000[tt, ...], vmin=286.6, vmax=287.6, origin='lower', cmap=cmaps.cmocean_matter)
    cax     = fig.add_axes([ax.get_position().x1+0.015, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar    = fig.colorbar(imT, cax=cax, extend='both')
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label(label='T (K)', fontsize=14)
    ax.set_xticks(np.arange(0, 128.1, 40))
    ax.set_xticklabels(np.arange(0, 128.1, 40)*0.2, fontsize=14)
    ax.set_xlabel('km', fontsize=14)
    ax.set_yticks(np.arange(0, 128.1, 40))
    ax.set_yticklabels(np.arange(0, 128.1, 40)*0.2, fontsize=14)
    ax.set_ylabel('km', fontsize=14)
    ax.set_title('Temp. Field @ 1000m', loc='left', fontsize=16)
    ax.set_title(time_now.strftime('%H:%M'), loc='right', fontsize=16)
    plt.savefig(f'/data/mlcloud/ch995334/hw1/T_animation/dt5_{time_now.strftime("%d%H%M")}.png', 
                facecolor='w', bbox_inches='tight', dpi=400)
    time_now= time_now+timedelta(minutes=1)

In [ ]:
level

### Test

In [ ]:
da_qv

In [ ]:
ds_dyn

In [ ]:
plt.plot(np.arange(da_dsw.shape[0]), da_dsw[:, 0, 0])
plt.xticks(np.arange(0, 721, 60))
plt.title('Downward SW @ TOA')

In [ ]:
plt.plot(np.arange(da_TH.shape[0]), da_TH[:, 0, 0]*1000)
plt.xticks(np.arange(0, 721, 60))
plt.title('Surface flux of potential temp.')

In [ ]:
plt.plot(np.arange(da_QV.shape[0]), da_QV[:, 0, 0])
plt.xticks(np.arange(0, 721, 60))
plt.title('Surface flux of water vapor')

In [ ]:
np.fromfile('../VVM/DATA/pbl_control/bar.dat')